In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install -U -q tensorflow-addons

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


In [ ]:
%pip install image-dataset-loader

In [ ]:
from image_dataset_loader import load

basepath="drive/MyDrive/Deepfakes/Data/Task_1"
train_data_dir = basepath + '/development'
validation_data_dir =  'drive/MyDrive/Deepfakes/Data/Task_2_3/evaluation'

# dimensions of our images.
img_width, img_height = 436, 500

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
# test_datagen = ImageDataGenerator(rescale=1. / 255)

data_with_aug = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=False,
                                   rescale=1./255,
                                  validation_split=0.2)

val = data_with_aug.flow_from_directory(validation_data_dir,
                                          class_mode="binary",
                                          target_size=(224, 224),
                                          batch_size=32,
                                          subset="validation"
                                          )

train = data_with_aug.flow_from_directory(train_data_dir,
                                          class_mode="binary",
                                          target_size=(224, 224),
                                          batch_size=32)


In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3))


In [ ]:
vgg16_model.output[-1]


In [ ]:
model = Sequential([vgg16_model,
                     tf.keras.layers.GlobalAveragePooling2D(),
                     Dense(512, activation = "relu"),
                     tf.keras.layers.BatchNormalization(),
                     Dense(128, activation = "relu"),
                     Dense(2, activation = "softmax")])
# We will be training only our dense layers not the entire VGG16 model
model.layers[0].trainable = False
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics="accuracy")

model.summary()

In [ ]:
history =  model.fit(train,
                    epochs=20
                    )

Epoch 1/20
24/24 [==============================] - 518s 21s/step - loss: 0.7622 - accuracy: 0.5289
Epoch 2/20
24/24 [==============================] - 499s 21s/step - loss: 0.5610 - accuracy: 0.7526
Epoch 3/20
24/24 [==============================] - 491s 20s/step - loss: 0.4756 - accuracy: 0.8158
Epoch 4/20
24/24 [==============================] - 481s 20s/step - loss: 0.4211 - accuracy: 0.8671
Epoch 5/20
24/24 [==============================] - 472s 20s/step - loss: 0.3710 - accuracy: 0.8697
Epoch 6/20
24/24 [==============================] - 473s 20s/step - loss: 0.3257 - accuracy: 0.9118
Epoch 7/20
24/24 [==============================] - 466s 19s/step - loss: 0.2955 - accuracy: 0.9211
Epoch 8/20
 2/24 [=>............................] - ETA: 7:36 - loss: 0.2901 - accuracy: 0.9375

In [ ]:
predictions = model.predict(val)


In [ ]:
scores = model.evaluate(val, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
model = Sequential([
    model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# We will be training only our dense layers not the entire VGG16 model
model.layers[-5].trainable = True
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics="accuracy")

model.summary()


In [ ]:
history = model.fit(train,
                    epochs=20,
                    validation_data=val)

# Calculate the loss and accuracy on the validation set
val_loss, val_acc = model.evaluate(val)
print('Validation loss:', val_loss)
print('Validation accuracy:', val_acc)

In [ ]:
predictions = model.predict(val)
scores = model.evaluate(val, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
_, _, val_labels = load(validation_data_dir, "inception", target_size=(224, 224), batch_size=32)


faces_pred = model.predict(val)
faces_pred = (np.amax(faces_pred, axis = 1)-0.5)*2
val_labels = np.argmax(val_labels, axis = 1)
print(faces_pred)


fpr, tpr, thresholds = roc_curve(val_labels, faces_pred)

# Compute area under the curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()